In [ ]:
from generate import *
from tree_search import *
from sklearn.externals import joblib

In [ ]:
class Network:
    def __init__(self, model):
        self.model = model
        self.board = np.zeros((15, 15))
    def make_move(self, *args):
        action = legal_nn_move(self.board, self.model)
        self.board[action] = -1.0 - 2*self.board.sum()
        return action
    def pass_move(self, action):
        self.board[action] = -1.0 - 2*self.board.sum()

In [ ]:
class Meatbag:
    def __init__(self):
        self.board = np.zeros((15, 15))
    def make_move(self, *args):
        while True:
            # many more possible problems
            raw = input()
            width = ord(raw[0].lower())
            if width > ord('i'):
                width -= 1
            width -= ord('a')
            height = int(raw[1:]) - 1
            if round(self.board[height, width]) == 0:
                break
            print('Wrong action')
        self.board[height, width] = -1.0 - 2*self.board.sum()
        return (height, width)
    
    def pass_move(self, action):
        self.board[action] = -1.0 - 2*self.board.sum()

In [ ]:
sl_policy = load_model('current_sgd_policy_model')
rl_policy = load_model('rl_policy_72%.keras')
value_policy = load_model('rl_value_model')
rollout_policy = joblib.load('logreg_3.pkl')

In [ ]:
board = np.zeros((15, 15))
last_action = (0, 0)
turn = 0

players = [Network(sl_policy),
           MonteCarloTreeSearch(black_starts=False, board, sl_policy, rollout_policy, value_policy, 0.5)]
# players = list(reversed(players))

while get_rollout_result(board, last_action) is None:
    show_board(board)
    last_action = players[turn % 2].make_move(500)
    players[(turn + 1) % 2].pass_move(last_action)
    board[last_action] = -1.0 + 2 * (turn % 2)
    turn += 1
show_board(board)
res = get_rollout_result(board, last_action)
if round(res) == 1:
    print('White won!')
elif round(res) == -1:
    print('Black won!')
else:
    print('Draw!')